<a href="https://colab.research.google.com/github/thiagolermen/machine-learning-portifolio/blob/main/src/algorithms/j-recurrent-neural-networks/PyTorch-bidirectional-lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bidirectional LSTM - PyTorch

## Imports

In [1]:
import torch
import torchvision
import torch.nn as nn # All nn modules
import torch.optim as optim # Optimization algorithms
import torch.nn.functional as F # Functions as loss and activation
import torchvision.datasets as datasets # Standard datasets
import torchvision.transforms as transforms # Transformations for the dataset
from torch.utils.data import DataLoader # Easier dataset managment

from tqdm import tqdm  # For a nice progress bar

In [2]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Model

In [3]:
# Hyperparameters
input_size = 28 #MNIST dataset
sequence_length = 28 # row by row
num_layers = 2
hidden_size = 256
num_classes = 10
learning_rate = 0.003
batch_size = 64
num_epochs = 3

In [4]:
# Recurrent neural network (many-to-one)
class BRNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(BRNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
    self.fc = nn.Linear(hidden_size*2, num_classes)

  def forward(self, x):
    # Set initial hidden and cell states
    h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)

    # Forward propagate LSTM
    out, _ = self.lstm(x, (h0, c0))

    # Decode the hidden state of the last time step
    out = self.fc(out[:, -1, :])
    return out

## Load data

In [5]:
train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root="dataset/", train=False, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw




Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw




Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw




Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## Train

In [6]:
# Initialize network (try out just using simple RNN, or GRU, and then compare with LSTM)
model = BRNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Network
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
    # Get data to cuda if possible
    data = data.to(device=device).squeeze(1)
    targets = targets.to(device=device)

    # forward
    scores = model(data)
    loss = criterion(scores, targets)

    # backward
    optimizer.zero_grad()
    loss.backward()

    # gradient descent update step/adam step
    optimizer.step()

100%|██████████| 938/938 [00:10<00:00, 89.00it/s]


## Test

In [7]:
def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0

  # Set model to eval
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device).squeeze(1)
      y = y.to(device=device)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

  # Toggle model back to train
  model.train()
  return num_correct / num_samples

print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 98.655006
Accuracy on test set: 98.37
